### Practical Part Assignment 1 [5 points]:

Deadline: 08.02.2024 12:00 (noon)

This assignment is designed to help you become acquainted with the basics of PyTorch (loading in data, training, testing). For those opting to work on personal devices, we encourage you to follow the provided setup instructions accompanying this assignment. Alternatively, Google Colab is also available as a convenient option.

### Instructions: 
1.  [3.5 points]: Please train a maximum likelihood classifier on CIFAR10 and report its out-of-sample (test set) accuracy. You are free to choose any architecture for the classifier; please provide a brief explanation of your choice. Below, you will find some starter code to assist you. If you are new to PyTorch, you might find it beneficial to refer to the tutorials available on the website [https://pytorch.org/tutorials/beginner/basics/intro.html](https://pytorch.org/tutorials/beginner/basics/intro.html).
2. [1.5 points]: Plot your classifier's calibration curve for a selected class vs. the remaining classes treated as a separate class.

### Additional Information:
To help you get a grip on tensor manipulations we uploaded an overview of the dimension and shape operation in PyTorch that we consider most valuable. Additionally, it includes a refresher on the concept of broadcasting. 


In [ ]:
# imports
import os

import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms

# set a global seed for Reproducibility 
torch.manual_seed(45)

# set system specific variables to make use of available hardware.
num_workers = os.cpu_count()
print('number of workers:', num_workers)

device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
print('device used:', device)

In [ ]:
# Path to the folder where the datasets are/should be downloaded (e.g. MNIST)
DATASET_PATH = "./data"

img_transform = transforms.Compose([
    transforms.ToTensor()
])

# load mnist dataset
test_dataset = CIFAR10(root=DATASET_PATH,
                       download=True,
                       train=False,
                       transform=img_transform)
print(f'size of test_dataset: {len(test_dataset)}')

train_dataset = CIFAR10(root=DATASET_PATH,
                        download=True,
                        train=True,
                        transform=img_transform)
print(f'size of train_dataset: {len(train_dataset)}')

In [15]:
batch_size = 10
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)